## Import libraries

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import time

## Use Colab env

In [ ]:
# Use google colab environments
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
print(tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Load dataset

In [ ]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
x_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 1))

In [ ]:
# train, validation split
# 5000 validation images
x_train,x_val,y_train, y_val = train_test_split(x_train, y_train, random_state=42, test_size=0.1) # train:test = 9:1

In [ ]:
# 10 class
CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# sample test image plot
images = x_test
labels = y_test

fig = plt.figure(figsize=(25, 4))

for i in np.arange(20):
  # row 2 column 10
  ax = fig.add_subplot(2, 10, i+1, xticks=[], yticks=[])
  plt.imshow(images[i])
  ax.set_title(CLASS_NAMES[labels[i].item()])

In [ ]:
# normalize the data
x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [ ]:
# convert labels to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
x_train.shape, y_train.shape

## Define AlexNet

In [ ]:
# define the AlexNet model
AlexNet = tf.keras.Sequential([
    tf.keras.layers.Resizing(256,256,interpolation = "bilinear", input_shape = x_train.shape[1:]), # resizing input images as 256*256
    tf.keras.layers.CenterCrop(224,224), # preprocessing layer
    tf.keras.layers.RandomFlip("horizontal"), # preprocessing layer
    tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=3, strides=2),
    tf.keras.layers.Conv2D(filters=256, kernel_size=5, strides=1, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=3, strides=2),
    tf.keras.layers.Conv2D(filters=384, kernel_size=3, strides=1, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(filters=384, kernel_size=3, strides=1, padding='same', activation='relu'),
    tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=3, strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

## Check structure of model

In [ ]:
AlexNet.summary() # check model structure

## Train the model

In [ ]:
# define the optimizer
optimizer = keras.optimizers.Adam(learning_rate=0.001)

# compile the model
AlexNet.compile(optimizer=optimizer,
              loss="categorical_crossentropy",
              metrics=["accuracy"])


In [ ]:
# define a callback to save the best model during training
checkpoint_path = "/content/gdrive/MyDrive/alexNet_tf.h5"
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_best_only=True)

In [ ]:
# train the model with validation set and save the best model
history = AlexNet.fit(x_train,y_train,epochs=10,batch_size=128,validation_data=(x_val, y_val), callbacks=[cp_callback], verbose = 1)

## Test the model

In [ ]:
# load the best model for testing
PATH =  "/content/gdrive/MyDrive/alexNet_tf.h5"
test_model = keras.models.load_model(PATH)

# evaluate the model on the test set
test_loss, test_acc = test_model.evaluate(x_test, y_test)

print(f"Test accuracy: {test_acc:.4f}")

## Plot predicted results with sample images

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)

  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(CLASS_NAMES[predicted_label],
                                100*np.max(predictions_array),
                                CLASS_NAMES[(true_label.astype(int))[0]]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = (true_label[i]).astype(int)[0]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
predictions = test_model.predict(x_test)

In [ ]:
# plot the first X test images, their predicted labels, and the true labels.
# color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], y_test, x_test)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], y_test)
plt.tight_layout()
plt.show()